In [1]:
!pip install pymysql

You should consider upgrading via the 'c:\users\trimu\miniconda3\envs\tf20\python.exe -m pip install --upgrade pip' command.


In [3]:
import pymysql.cursors

host = 'localhost'
user_id = 'root'
pswd = 'Password12345'
db = 'test_db'

try:
    con = pymysql.connect(host=host, user=user_id, password=pswd, db=db, charset='utf8mb4')
    print('connect success')
except Exception as e:
    print(e)

connect success


In [5]:
sql = "select F_NAME, L_NAME, START_DATE from employees join job_history on EMP_ID = EMPL_ID where dep_id = '5';"

In [6]:
cur = con.cursor()

cur.execute(sql)
cur.fetchall()

(('Alice', 'James', datetime.date(2001, 8, 1)),
 ('Steve', 'Wells', datetime.date(2001, 8, 16)),
 ('Santosh', 'Kumar', datetime.date(2000, 8, 16)),
 ('Ann', 'Jacob', datetime.date(2016, 8, 16)))

In [2]:
try:        
    cur.execute("DROP TABLE IF EXISTS Counts")
    cur.execute("CREATE TABLE Counts (email TEXT, count INTEGER)")
    print ('table created successfully')
except Exception as e:
    print ('error in operation, ', e)
    con.rollback()
    con.close()

table created successfully


- `From: ` 으로 시작하는 record 에서 email id 를 추출하여 db 에 건수 누적 

- SQL 문에서 `?` 는 placeholder 를 의미. 여러개가 올 수 있으므로 tuple 로 mapping 시킨다

In [3]:
fname = 'emailbox-short.txt'
fh = open(fname)

for line in fh:
    if not line.startswith('From: '):
        continue
    pieces = line.split()
    email = pieces[1]
    cur.execute('SELECT count FROM Counts WHERE email = ?', (email,))
    row = cur.fetchone()
    if row is None:
        cur.execute('INSERT INTO Counts (email, count) VALUES (?, 1)', 
                        (email,))
    else:
        cur.execute('UPDATE Counts SET count = count + 1 WHERE email = ?', 
                        (email,))
        
    con.commit()

### 입력된 data 확인

In [4]:
sqlstr = 'SELECT email, count FROM Counts ORDER BY count DESC LIMIT 10'

for row in cur.execute(sqlstr):
    print(row[0], row[1])

cwen@iupui.edu 5
zqian@umich.edu 4
david.horwitz@uct.ac.za 4
louis@media.berkeley.edu 3
gsilver@umich.edu 3
stephen.marquard@uct.ac.za 2
rjlowe@iupui.edu 2
wagnermr@iupui.edu 1
antranig@caret.cam.ac.uk 1
gopal.ramasammycook@gmail.com 1


In [5]:
cur.execute('SELECT email, count FROM Counts')
cur.fetchall()

[('stephen.marquard@uct.ac.za', 2),
 ('louis@media.berkeley.edu', 3),
 ('zqian@umich.edu', 4),
 ('rjlowe@iupui.edu', 2),
 ('cwen@iupui.edu', 5),
 ('gsilver@umich.edu', 3),
 ('wagnermr@iupui.edu', 1),
 ('antranig@caret.cam.ac.uk', 1),
 ('gopal.ramasammycook@gmail.com', 1),
 ('david.horwitz@uct.ac.za', 4),
 ('ray@media.berkeley.edu', 1)]

In [6]:
cur.execute('SELECT email, count FROM Counts')
cur.fetchone()

('stephen.marquard@uct.ac.za', 2)

In [7]:
cur.fetchone()

('louis@media.berkeley.edu', 3)

In [8]:
cur.fetchone()

('zqian@umich.edu', 4)

In [9]:
cur.fetchall()

[('rjlowe@iupui.edu', 2),
 ('cwen@iupui.edu', 5),
 ('gsilver@umich.edu', 3),
 ('wagnermr@iupui.edu', 1),
 ('antranig@caret.cam.ac.uk', 1),
 ('gopal.ramasammycook@gmail.com', 1),
 ('david.horwitz@uct.ac.za', 4),
 ('ray@media.berkeley.edu', 1)]

In [10]:
sql = 'INSERT INTO Counts (email, count) VALUES (?, ?);'

In [11]:
emails=[('test1@test.com', 100), 
             ('test2@test.com', 200), 
             ('test3@test.com', 300)]

cur.executemany(sql, emails)

In [12]:
cur.execute('SELECT email, count FROM Counts')
cur.fetchall()

[('stephen.marquard@uct.ac.za', 2),
 ('louis@media.berkeley.edu', 3),
 ('zqian@umich.edu', 4),
 ('rjlowe@iupui.edu', 2),
 ('cwen@iupui.edu', 5),
 ('gsilver@umich.edu', 3),
 ('wagnermr@iupui.edu', 1),
 ('antranig@caret.cam.ac.uk', 1),
 ('gopal.ramasammycook@gmail.com', 1),
 ('david.horwitz@uct.ac.za', 4),
 ('ray@media.berkeley.edu', 1),
 ('test1@test.com', 100),
 ('test2@test.com', 200),
 ('test3@test.com', 300)]

## 연습문제

- 다음 table 을 database 로 생성  

- executemany( ) 를 이용하여 A1~A3 까지의 data 를 일괄 insert

- 전체를 조회  

- A4 를 추가로 insert  

- A3 의 나이를 59 로 변경  

- A1 을 삭제  

- 전체를 조회


|학생번호| 성명 |나이 |성적|
|:------:|:-----:|---|----|
|A1|홍길동|20|70|
|A2|유병길|16|80|
|A3|김길수|29|90|
|A4|박재만|25|86|

In [11]:
import sqlite3

con = sqlite3.connect('test.sqlite')

In [12]:
try:
    cur = con.cursor()
    cur.execute("DROP TABLE IF EXISTS student")
    cur.execute("""CREATE TABLE student (
                   studentID CHAR(2),
                   name TEXT (20) NOT NULL,
                   age INTEGER,
                   score REAL
                    );""")
except Exception as e:
    print(e)

In [13]:
cur = con.cursor()

sql = "INSERT INTO student (studentID, name,  age, score) VALUES(?, ?, ?, ?);"
student_data = [("A1", "홍길동", 20, 70), ("A2", "유병길", 16, 80), ("A3", "김길수", 29, 90)]
cur.executemany(sql, student_data)

In [14]:
sql = "SELECT * FROM student;"
cur.execute(sql)

In [15]:
cur.fetchall()

[('A1', '홍길동', 20, 70.0), ('A2', '유병길', 16, 80.0), ('A3', '김길수', 29, 90.0)]

In [16]:
sql = "INSERT INTO student (studentID, name,  age, score) VALUES('A4', '박재만', 26, 70);"

In [17]:
cur.execute(sql)

In [18]:
sql = "SELECT * FROM student;"
cur.execute(sql)
cur.fetchall()

[('A1', '홍길동', 20, 70.0),
 ('A2', '유병길', 16, 80.0),
 ('A3', '김길수', 29, 90.0),
 ('A4', '박재만', 26, 70.0)]

In [19]:
cur.execute("UPDATE student SET age=59 WHERE studentID = 'A3'")

In [20]:
sql = "SELECT * FROM student;"
cur.execute(sql)
cur.fetchall()

[('A1', '홍길동', 20, 70.0),
 ('A2', '유병길', 16, 80.0),
 ('A3', '김길수', 59, 90.0),
 ('A4', '박재만', 26, 70.0)]

In [22]:
cur.execute("DELETE FROM student WHERE studentID = 'A1'")

In [23]:
sql = "SELECT * FROM student;"
cur.execute(sql)
cur.fetchall()

[('A2', '유병길', 16, 80.0), ('A3', '김길수', 59, 90.0), ('A4', '박재만', 26, 70.0)]